In [ ]:
import gdal
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio
import subprocess
import pandas as pd
import os, glob
from rasterio import plot
import geopandas as gpd


################ NOW FUNCTIONS  ###################

##------------------
# Functions used in the script 
##------------------

def create_dir_and_check_existence(path):
    #Create a new directory
    try:
        os.makedirs(path)
    except:
        print ("directory already exists")

def open_image(url):
    image_data = open_http_query(url)
    
    if not image_data:
            return None
            
    mmap_name = "/vsimem/"+uuid4().get_hex()
    gdal.FileFromMemBuffer(mmap_name, image_data.read())
    gdal_dataset = gdal.Open(mmap_name)
    image = gdal_dataset.GetRasterBand(1).ReadAsArray()
    gdal_dataset = None
    gdal.Unlink(mmap_name)
    
    return image

############################################################################
#####  Parameters and argument set up ########### 

#ARGS 1
in_dir = "/nfs/bparmentier-data/Data/projects/urban_green_planning/GAstart"
#in_dir <- "/nfs/tjovanovic-data/Data/Baltimore/Hydrology/GAstart"
#ARGS 2
out_dir = "/nfs/bparmentier-data/Data/projects/urban_green_planning/outputs"
#ARGS 3:
create_out_dir=True #create a new ouput dir if TRUE
#ARGS 7
out_suffix = "processing_data_11082018" #output suffix for the files and ouptut folder
#ARGS 8
num_cores = 2 # number of cores
file_format = ".tif"

dem_baltimore_filename = "DEM_BaltArea_1m.tif"
lc_baltimore_filename = "landCover_area1m.tif"
reg_outline_filename = "watersheds8digit.shp"

################# START SCRIPT ###############################

######### PART 0: Set up the output dir ################

#set up the working directory
#Create output directory

if create_out_dir==True:
    #out_path<-"/data/project/layers/commons/data_workflow/output_data"
    out_dir = "output_data_"+out_suffix
    out_dir = os.path.join(in_dir,out_dir)
    create_dir_and_check_existence(out_dir)
    os.chdir(out_dir)        #set working directory
else:
    os.chdir(create_out_dir) #use working dir defined earlier

In [ ]:
#######################################
### PART 1: Read in DATA #######

dem_baltimore_filename = os.path.join(in_dir,dem_baltimore_filename)
lc_baltimore_filename = os.path.join(in_dir,lc_baltimore_filename)
reg_outline_filename = os.path.join(in_dir,reg_outline_filename)

reg_gpd = gpd.read_file(reg_outline_filename)
reg_gpd.describe()
reg_gpd.plot(column="mde8name")
reg_gpd.head() 





In [ ]:
#########################################
### PART 2: Crop to region of interest #######
#Open existing raster ds

dem_baltimore = rasterio.open(os.path.join(in_dir,dem_baltimore_filename))
reg_dem_gpd = reg_gpd.to_crs(dem_baltimore.crs)

extent_val = reg_dem_gpd.total_bounds #minx,MINY,MAXX,MAXY
#Need a gdal object
dem_baltimore = gdal.Open(os.path.join(in_dir,dem_baltimore_filename))

#gdal.WarpOptions()
gdal.Warp("cropped_dem.tif",dem_baltimore,outputBounds=extent_val)
dem_baltimore= None

#lc_baltimore = gdal.Open(os.path.join(in_dir,lc_baltimore_filename))
lc_baltimore = gdal.Open(os.path.join(in_dir,lc_baltimore_filename))

lc_baltimore = rasterio.open(os.path.join(in_dir,lc_baltimore_filename))

reg_lc_gpd = reg_gpd.to_crs(lc_baltimore.crs)

extent_val = reg_lc_gpd.total_bounds #minx,MINY,MAXX,MAXY

gdal.Warp("cropped_lc.tif",lc_baltimore,outputBounds=extent_val)
lc_baltimore= None

In [ ]:
#########################
### PART 3: reproject raster to Maryland State plane #######

#https://gis.stackexchange.com/questions/257257/how-to-use-gdal-warp-cutline-option/257292
#https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html#get-raster-metadata

#local md projection:
crs_reg = "+proj=lcc +lat_1=39.45 +lat_2=38.3 +lat_0=37.66666666666666 +lon_0=-77 +x_0=400000 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs" 

#gdal.WarpOptions()
dem_cropped_baltimore = os.path.join(out_dir,"cropped_dem_.tif")
gdal.Warp("dem_md.tif", dem_cropped_baltimore,)
dem_baltimore= None


#r_dem_md <- projectRaster(r_dem_crop,res=c(1,1),crs=crs_reg)
#writeRaster(r_dem_md,"dem_md.tif")

#r_lc_md <- projectRaster(r_lc_crop,r_dem_md)

#### Use gdal:reproject dem first 

#data_type_str <- dataType(r_dem)
#NAvalue(r_dem)
#dataType_selected <- dataType_table$r_type==data_type_str
#data_type_table_selected <- dataType_table[dataType_selected,]
#data_type_table_selected

#dem_md_baltimore <- file.path(out_dir,paste0("r_dem_crop_","md_",out_suffix,file_format))
#src_dataset <- dem_crop_baltimore_filename 
#dst_dataset <- dem_md_baltimore
  
output_type <- data_type_table_selected$gdal_type
NA_flag_val_str <- data_type_table_selected$min
CRS_reg <- crs_reg



#gdalwarp -t_srs '+proj=utm +zone=11 +datum=WGS84' -overwrite raw_spot.tif utm11.tif
gdal.Warp()
cmd_str = paste0("gdalwarp",
                 " -ot ",output_type,
                 " -srcnodata ",NA_flag_val_str,
                 " -t_srs ", paste0("'",CRS_reg,"'"),
                 " -dstnodata ",NA_flag_val_str,
                 " -overwrite",
                 " ",src_dataset, 
                 " ",dst_dataset)             

system(cmd_str)